In [ ]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
from time import sleep

In [ ]:
folder = r"C:\python_web_scraping_homework\project_gutenberg"
os.makedirs(folder, exist_ok=True)

In [ ]:
def extract_book_body(text, title):
    pattern = rf"\*\*\* START OF THE PROJECT GUTENBERG EBOOK {title} \*\*\*(.*?)\*\*\* END OF THE PROJECT GUTENBERG EBOOK {title} \*\*\*"
    match = re.search(pattern, text, flags=re.DOTALL | re.IGNORECASE)
    if not match:
        return ""
    content = match.group(1).strip()
    lines = content.splitlines()
    return '\n'.join(re.sub(r'[A-Za-z]', '', line) for line in lines)

def sanitize_filename(name):
    return re.sub(r'[\\/:*?"<>|]', "", name)

In [ ]:
url = "https://www.gutenberg.org/browse/languages/zh"
base_url = "https://www.gutenberg.org"
headers = {"User-Agent": "Mozilla/5.0"}

res = requests.get(url)
soup = BeautifulSoup(res.text, "html.parser")

link_tags = soup.select("li.pgdbetext > a[href]")

book_urls = [base_url + tag['href'] for tag in link_tags]
book_titles = [tag.text.strip() for tag in link_tags]

In [ ]:
index = 0
target = 200

while len([f for f in os.listdir(folder) if f.endswith('.txt')]) < target and index < len(book_titles):
    title = book_titles[index]
    book_url = book_urls[index]
    index += 1

    try:
        if re.search(r'[A-Za-z]', title):
            continue

        print(f"正在處理第 {index} 本書：{title}")
        res = requests.get(book_url, headers=headers, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")
        tag = soup.find("a", string="Plain Text UTF-8")
        if not tag or not tag.has_attr("href"):
            continue

        txt_url = base_url + tag["href"]
        txt_res = requests.get(txt_url, headers=headers, timeout=10)

        book_body = extract_book_body(txt_res.text, title)
        if len(book_body) < 100:
            continue

        filename = os.path.join(folder, f"{sanitize_filename(title)}.txt")
        with open(filename, "w", encoding="utf-8") as f:
            f.write(book_body)

        sleep(1)

    except Exception as e:
        print(f"錯誤：{e}")
        continue

print(f"已完成，寫入檔案數：{len([f for f in os.listdir(folder) if f.endswith('.txt')])}")